# Polaris-Arctic Demo Dataset

1. ERA5 Reanalysis dataset
    - Sea Ice Concentration
    - 10m Wind Speed (u, v)
    - Significant Wave Height
2. ArcNet
    - Priority Areas for Conservation

## ERA5 Reanalysis Dataset

To download this dataset, you need to sign in [Climate Data Source](https://cds.climate.copernicus.eu/) and save personal access token into __$HOME/.cdsapirc__. 

Follow the detail instruction [here](https://cds.climate.copernicus.eu/how-to-api).

In [7]:
import cdsapi

In [8]:
dataset = "reanalysis-era5-single-levels"
request = {
    "product_type": ["reanalysis"],
    "variable": [
        "10m_u_component_of_wind",
        "10m_v_component_of_wind",
        "significant_height_of_combined_wind_waves_and_swell",
        "sea_ice_cover"
    ],
    "year": ["2024"],
    "month": ["10"],
    "day": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12",
        "13", "14", "15",
        "16", "17", "18",
        "19", "20", "21",
        "22", "23", "24",
        "25", "26", "27",
        "28", "29", "30",
        "31"
    ],
    "time": ["09:00"],
    "data_format": "netcdf",
    "download_format": "unarchived",
    "area": [90, -180, 45, 180]
}

client = cdsapi.Client()
client.retrieve(dataset, request).download(target='./dataset/era5/tmp.zip')

2025-02-15 14:42:03,720 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-02-15 14:42:03,722 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-02-15 14:42:04,163 INFO Request ID is db4d1b43-349f-4c82-9806-86af3f3c62d6
2025-02-15 14:42:04,241 INFO status has been updated to accepted
2025-02-15 14:42:12,732 INFO status has been updated to successful


'./dataset/era5/tmp.zip'

In [9]:
! unzip -o ./dataset/era5/tmp.zip -d ./dataset/era5
! rm ./dataset/era5/tmp.zip

Archive:  ./dataset/era5/tmp.zip
  inflating: ./dataset/era5/data_stream-oper_stepType-instant.nc  
  inflating: ./dataset/era5/data_stream-wave_stepType-instant.nc  


## ArcNet Dataset

Reference.

James, T.D., Sommerkorn, M., Solovyev, B. et al. Whole-ocean network design and implementation pathway for Arctic marine conservation. npj Ocean Sustain 3, 25 (2024). https://doi.org/10.1038/s44183-024-00047-9

In [10]:
! wget -O ./dataset/arcnet/tmp.zip \
    https://static-content.springer.com/esm/art%3A10.1038%2Fs44183-024-00047-9/MediaObjects/44183_2024_47_MOESM6_ESM.zip

--2025-02-15 14:42:18--  https://static-content.springer.com/esm/art%3A10.1038%2Fs44183-024-00047-9/MediaObjects/44183_2024_47_MOESM6_ESM.zip
Resolving static-content.springer.com (static-content.springer.com)... 151.101.192.95, 151.101.128.95, 151.101.64.95, ...
Connecting to static-content.springer.com (static-content.springer.com)|151.101.192.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3709648 (3.5M) [application/octet-stream]
Saving to: ‘./dataset/arcnet/tmp.zip’

./dataset/arcnet/tm 100%[===================>]   3.54M  2.58MB/s    in 1.4s    

2025-02-15 14:42:20 (2.58 MB/s) - ‘./dataset/arcnet/tmp.zip’ saved [3709648/3709648]



In [11]:
! unzip -o ./dataset/arcnet/tmp.zip -d ./dataset/arcnet
! rm ./dataset/arcnet/tmp.zip

Archive:  ./dataset/arcnet/tmp.zip
 extracting: ./dataset/arcnet/ArcNet_shp_files/ArcNet_Domain_curved.cpg  
  inflating: ./dataset/arcnet/ArcNet_shp_files/ArcNet_Domain_curved.dbf  
  inflating: ./dataset/arcnet/ArcNet_shp_files/ArcNet_Domain_curved.prj  
  inflating: ./dataset/arcnet/ArcNet_shp_files/ArcNet_Domain_curved.qmd  
  inflating: ./dataset/arcnet/ArcNet_shp_files/ArcNet_Domain_curved.shp  
  inflating: ./dataset/arcnet/ArcNet_shp_files/ArcNet_Domain_curved.shx  
 extracting: ./dataset/arcnet/ArcNet_shp_files/ArcNet_Domain_uncurved.cpg  
  inflating: ./dataset/arcnet/ArcNet_shp_files/ArcNet_Domain_uncurved.dbf  
  inflating: ./dataset/arcnet/ArcNet_shp_files/ArcNet_Domain_uncurved.prj  
  inflating: ./dataset/arcnet/ArcNet_shp_files/ArcNet_Domain_uncurved.qmd  
  inflating: ./dataset/arcnet/ArcNet_shp_files/ArcNet_Domain_uncurved.shp  
  inflating: ./dataset/arcnet/ArcNet_shp_files/ArcNet_Domain_uncurved.shx  
 extracting: ./dataset/arcnet/ArcNet_shp_files/ArcNet_PACs_curved

Convert the ArcNet Shape file(Vector) into the Raster(Pixels)

In [12]:
! gdal_rasterize -l ArcNet_PACs_uncurved \
    -a PAC_ID -tr 1000.0 1000.0 -a_nodata 0.0 -ot Float32 -of GTiff \
        ./dataset/arcnet/ArcNet_shp_files/ArcNet_PACs_uncurved.shp \
            ./dataset/arcnet/arcnet_rasterized_1km.tif

0...10...20...30...40...50...60...70...80...90...100 - done.


Reproject to the coordinate system EPSG:4326 with 0.25°x0.25° resolution.

In [13]:
! gdalwarp -overwrite -t_srs EPSG:4326 -tr 0.25 0.25 -r near \
    -te -180.0 45.0 180.0 90.0 -te_srs EPSG:4326 -of GTiff \
        ./dataset/arcnet/arcnet_rasterized_1km.tif ./dataset/arcnet/arcnet_epsg4326_25e-2.tif

Creating output file that is 1440P x 180L.
Processing ./dataset/arcnet/arcnet_rasterized_1km.tif [1/1] : 0Using internal nodata values (e.g. 0) for image ./dataset/arcnet/arcnet_rasterized_1km.tif.
Copying nodata values from source ./dataset/arcnet/arcnet_rasterized_1km.tif to destination ./dataset/arcnet/arcnet_epsg4326_25e-2.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


# 